In [ ]:
# ---------------------
# Question 2
# ---------------------

In [ ]:
# ---------------------
# Part e
# ---------------------

%pip install networkx

import pandas as pd
import numpy as np
from scipy.stats import norm
import networkx as nx
from networkx.algorithms.approximation import greedy_tsp

# ---------------------
# STEP 1: Load Datasets
# ---------------------

# Load randomness data: probabilities and demand distributions
randomness_df = pd.read_csv("/Users/alvina/Downloads/randomness.csv")
station_names = randomness_df['Unnamed: 0'].values
probabilities = randomness_df['Probability'].values
means = randomness_df['Mean_Demand'].values
stds = randomness_df['Std_Dev_Demand'].values

# Load travel cost matrix (Station_0 is the depot)
costs_df = pd.read_csv("/Users/alvina/Downloads/costs.csv")
cost_matrix = costs_df.iloc[:, 1:].values  # Remove station name column

# -------------------------------
# STEP 2: Helper Functions
# -------------------------------

# Simulate one scenario: which stations need fuel and how much
def simulate_scenario(probabilities, means, stds):
    active = np.random.binomial(1, probabilities)  # 0 = inactive, 1 = active
    demands = np.where(active == 1, norm.rvs(loc=means, scale=stds), 0)
    return active, np.maximum(demands, 0)  # Ensure no negative demand

# Compute approximate TSP cost using NetworkX's greedy algorithm
def compute_travel_cost(active_nodes, cost_matrix):
    if len(active_nodes) == 0:
        return 0  # No active stations
    if len(active_nodes) == 1:
        # Route: depot -> station -> depot
        return cost_matrix[0, active_nodes[0]+1] + cost_matrix[active_nodes[0]+1, 0]

    # Create graph of active nodes + depot (index 0)
    G = nx.Graph()
    nodes = [0] + [i + 1 for i in active_nodes]  # Shift by 1 due to cost matrix layout
    for i in nodes:
        for j in nodes:
            if i != j:
                G.add_edge(i, j, weight=cost_matrix[i, j])

    tsp_route = greedy_tsp(G, weight="weight")
    tsp_route.append(tsp_route[0])  # Complete the round-trip
    total_cost = sum(cost_matrix[tsp_route[i], tsp_route[i+1]] for i in range(len(tsp_route)-1))
    return total_cost

# -------------------------------
# STEP 3: Run SAA Simulation
# -------------------------------

# Define range of truck sizes to test
truck_sizes = np.arange(500, 5000, 500)

n_trials = 20
scenarios_per_trial = 10
results = []

for truck_size in truck_sizes:
    trial_costs = []

    for _ in range(n_trials):
        scenario_costs = []

        for _ in range(scenarios_per_trial):
            # Simulate a single scenario
            active, demands = simulate_scenario(probabilities, means, stds)
            active_indices = np.where(active == 1)[0]
            total_demand = np.sum(demands)

            # Mis-sizing penalty
            if truck_size > total_demand:
                mis_cost = 0.09 * (truck_size - total_demand)
            else:
                mis_cost = 0.13 * (total_demand - truck_size)

            # Travel cost for active stations
            travel_cost = compute_travel_cost(active_indices, cost_matrix)

            # Total scenario cost
            scenario_total_cost = mis_cost + travel_cost
            scenario_costs.append(scenario_total_cost)

        # Average cost for the trial
        trial_costs.append(np.mean(scenario_costs))

    # Average of all trials for current truck size
    mean_cost = np.mean(trial_costs)
    std_dev = np.std(trial_costs)
    results.append({
        "Truck_Size": truck_size,
        "Expected_Cost": mean_cost,
        "Std_Dev": std_dev
    })

# Convert results to DataFrame for analysis
saa_df = pd.DataFrame(results)

# Identify the best truck size with the lowest expected cost
best_row = saa_df.loc[saa_df['Expected_Cost'].idxmin()]
print(" Optimal Expected Cost:", round(best_row['Expected_Cost'], 2))
print(" Best Truck Size:", int(best_row['Truck_Size']), "liters")


2837.82s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
 Optimal Expected Cost: 253.33
 Best Truck Size: 2500 liters


In [ ]:
# ---------------------
# Part f
# ---------------------

# Define costs as a DataFrame for travel cost lookup
costs = costs_df.set_index('Unnamed: 0')

# Generate a random demand scenario
def generate_scenario():
    scenario = {}
    for _, row in randomness_df.iterrows():
        station = row['Unnamed: 0']
        if np.random.rand() < row['Probability']:
            demand = np.random.normal(row['Mean_Demand'], row['Std_Dev_Demand'])
            scenario[station] = max(0, demand)
    return scenario

# Estimate cost for a given scenario using naive truck size and route
def solve_scenario(demand_scenario):
    total_demand = sum(demand_scenario.values())
    truck_size = total_demand  # naive: match the day's demand

    # Penalty costs
    underage = max(0, total_demand - truck_size)
    overage = max(0, truck_size - total_demand)
    penalty_cost = under_cost * underage + over_cost * overage

    # Naive route: from storage to each station and back (chain-style)
    nodes = ['Station_0'] + list(demand_scenario.keys()) + ['Station_0']
    travel_cost = 0
    for i in range(len(nodes) - 1):
        src = nodes[i]
        dst = nodes[i + 1]
        if src != dst:
            travel_cost += costs.loc[src, dst]

    return travel_cost + penalty_cost

# Redefine solve_perfect_foresight: optimize based on known demand
def solve_perfect_foresight(scenario):
    # Route and truck size are customized to known demand
    total_demand = sum(scenario.values())
    truck_size = total_demand  # perfect foresight, no overage or underage
    penalty_cost = 0  # no penalty since truck size matches demand

    # Same naive routing logic as before
    nodes = ['Station_0'] + list(scenario.keys()) + ['Station_0']
    travel_cost = 0
    for i in range(len(nodes) - 1):
        src = nodes[i]
        dst = nodes[i + 1]
        if src != dst:
            travel_cost += costs.loc[src, dst]
    
    return travel_cost + penalty_cost

# Run Monte Carlo for WS value
ws_costs = []
for _ in range(n_trials):
    trial_costs = []
    for _ in range(scenarios_per_trial):
        scenario = generate_scenario()
        cost = solve_perfect_foresight(scenario)
        trial_costs.append(cost)
    ws_costs.append(np.mean(trial_costs))

expected_WS = np.mean(ws_costs)

# Calculate EVPI (Expected Value of Perfect Information)
EVPI = expected_WS - optimal_expected_cost  # optimal_expected_cost is from part (e)

print(f"Expected Wait-and-See (WS) Cost: ${expected_WS:.2f}")
print(f"Expected Value of Perfect Information (EVPI): ${EVPI:.2f}")


Expected Wait-and-See (WS) Cost: $276.56
Expected Value of Perfect Information (EVPI): $13.75


In [ ]:
# ---------------------
# Part g
# ---------------------

# Import necessary libraries
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.approximation import greedy_tsp

# Calculate penalty costs based on mismatch between truck size and total demand
# If the truck is larger than the demand, it's an overage (under-cost penalty)
# If the truck is smaller than the demand, it's an underage (over-cost penalty)

# Define constants
under_cost = 0.13  # Cost per unit of underage
over_cost = 0.09   # Cost per unit of overage
n_trials = 20
scenarios_per_trial = 10

# Load costs and randomness data
costs_df = pd.read_csv('costs.csv')
randomness_df = pd.read_csv('randomness.csv')

# Define the mapping of station names to indices for the cost matrix
station_index_mapping = {station: idx for idx, station in enumerate(costs_df.columns[1:])}

# Generate a random demand scenario (simulate demand using probabilities and distributions)
def generate_scenario():
    scenario = {}
    for _, row in randomness_df.iterrows():
        station = row['Unnamed: 0']
        if np.random.rand() < row['Probability']:
            demand = np.random.normal(row['Mean_Demand'], row['Std_Dev_Demand'])
            scenario[station] = max(0, demand)
    return scenario

# Compute travel cost using greedy TSP algorithm (optimized route)
def compute_travel_cost(active_nodes, cost_matrix):
    if len(active_nodes) == 0:
        return 0  # No active stations
    if len(active_nodes) == 1:
        return cost_matrix[0, active_nodes[0] + 1] + cost_matrix[active_nodes[0] + 1, 0]

    # Convert active node names to indices
    active_node_indices = [station_index_mapping[station] for station in active_nodes]
    
    # Create graph of active nodes + depot (index 0)
    G = nx.Graph()
    nodes = [0] + active_node_indices  # Include depot (index 0)
    for i in nodes:
        for j in nodes:
            if i != j:
                G.add_edge(i, j, weight=cost_matrix[i, j])

    tsp_route = greedy_tsp(G, weight="weight")
    tsp_route.append(tsp_route[0])  # Complete the round-trip
    total_cost = sum(cost_matrix[tsp_route[i], tsp_route[i + 1]] for i in range(len(tsp_route) - 1))
    return total_cost

# Step 1: Define the EV solution (visit all, use mean demand as truck size)
mean_demands = randomness_df.set_index('Unnamed: 0')['Mean_Demand'].to_dict()
ev_truck_size = sum(mean_demands.values())  # Truck size = total expected demand
ev_route = ['Station_0'] + list(mean_demands.keys()) + ['Station_0']  # Route includes depot and all stations

# Step 2: Solve for EEV using fixed truck size (mean demand)
def solve_EEV_using_fixed_truck(ev_truck_size, scenario):
    # Penalty cost based on mismatch with realized total demand
    total_demand = sum(scenario.values())
    underage = max(0, total_demand - ev_truck_size)
    overage = max(0, ev_truck_size - total_demand)
    penalty_cost = under_cost * underage + over_cost * overage

    # Travel cost using the optimized route (greedy TSP)
    active_stations = [station for station, demand in scenario.items() if demand > 0]
    travel_cost = compute_travel_cost(active_stations, cost_matrix)

    return travel_cost + penalty_cost

# Step 3: Run EEV simulation over 20 trials and 10 scenarios per trial
eev_costs = []
for _ in range(n_trials):
    trial_costs = []
    for _ in range(scenarios_per_trial):
        scenario = generate_scenario()
        cost = solve_EEV_using_fixed_truck(ev_truck_size, scenario)
        trial_costs.append(cost)
    eev_costs.append(np.mean(trial_costs))

# Calculate the expected cost of the EEV solution
expected_EEV = np.mean(eev_costs)

# Step 4: Calculate VSS (Value of the Stochastic Solution)
# Load optimal expected cost from part (e) for comparison
optimal_expected_cost = 262.81  # Use the value from part (e)

VSS = expected_EEV - optimal_expected_cost  # Value of the stochastic solution

# Output the results
print(f"Expected cost of EEV solution: ${expected_EEV:.2f}")
print(f"Value of the Stochastic Solution (VSS): ${VSS:.2f}")


Expected cost of EEV solution: $355.67
Value of the Stochastic Solution (VSS): $92.86
